In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point, Polygon
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
import sklearn
import sklearn.utils
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import scipy.stats as stats
from datetime import datetime
import datetime
from datetime import timedelta

In [ ]:
"""
Importing csv file with new data and filtering it further.

    Returns:
    _ _ _ _ _ _ _ _ _ _ _ _ _
    Dataframe with dropped nan values and dropped index.
    
"""
newdata = pd.read_csv("/work/Data/mergeddata.csv")
newdata = newdata.dropna().drop(columns=["Unnamed: 0"])
newdata

,Unnamed: 0.1,timestamp,vin,sequence,navigation_location_coordinate_latitude,navigation_location_coordinate_longitude,key
0,138.0,1578942333714,00089af9f7d3588a888379cf0ffc53cc,733,41.492962,-81.995163,eyJ0aW1lc3RhbXAiOjE1Nzg5NDIzMzM3MTQsInZpbiI6Ij...
1,3548.0,1578917535963,005b420fd9275f080ad79e1c4330357e,1142,41.335046,-81.398117,eyJ0aW1lc3RhbXAiOjE1Nzg5MTc1MzU5NjMsInZpbiI6Ij...
2,3549.0,1578917659974,005b420fd9275f080ad79e1c4330357e,1142,41.337986,-81.402637,eyJ0aW1lc3RhbXAiOjE1Nzg5MTc2NTk5NzQsInZpbiI6Ij...
3,3550.0,1578917668955,005b420fd9275f080ad79e1c4330357e,1142,41.338722,-81.403610,eyJ0aW1lc3RhbXAiOjE1Nzg5MTc2Njg5NTUsInZpbiI6Ij...
4,3556.0,1578918206960,005b420fd9275f080ad79e1c4330357e,1142,41.292217,-81.409466,eyJ0aW1lc3RhbXAiOjE1Nzg5MTgyMDY5NjAsInZpbiI6Ij...
...,...,...,...,...,...,...,...
736058,235.0,1580774398987,cc63c9825a40f6dc03a7ccfc6cdc7e8d,623,40.089127,-82.838978,eyJ0aW1lc3RhbXAiOjE1ODA3NzQzOTg5ODcsInZpbiI6Im...
736059,238.0,1580774399993,cc63c9825a40f6dc03a7ccfc6cdc7e8d,623,40.089245,-82.838651,eyJ0aW1lc3RhbXAiOjE1ODA3NzQzOTk5OTMsInZpbiI6Im...
736060,241.0,1580774398878,d8538c269b6cb7eec9a1f3b2bd8fc2d7,764,40.304051,-83.524914,eyJ0aW1lc3RhbXAiOjE1ODA3NzQzOTg4NzgsInZpbiI6Im...
736061,242.0,1580774399879,d8538c269b6cb7eec9a1f3b2bd8fc2d7,764,40.304008,-83.524871,eyJ0aW1lc3RhbXAiOjE1ODA3NzQzOTk4NzksInZpbiI6Im...


In [ ]:
"""

Making sure there are no duplicated values in the dataframe.

"""
newdata[newdata.duplicated()]

,Unnamed: 0.1,timestamp,vin,sequence,navigation_location_coordinate_latitude,navigation_location_coordinate_longitude,key
352151,138.0,1578942333714,00089af9f7d3588a888379cf0ffc53cc,733,41.492962,-81.995163,eyJ0aW1lc3RhbXAiOjE1Nzg5NDIzMzM3MTQsInZpbiI6Ij...
352152,3548.0,1578917535963,005b420fd9275f080ad79e1c4330357e,1142,41.335046,-81.398117,eyJ0aW1lc3RhbXAiOjE1Nzg5MTc1MzU5NjMsInZpbiI6Ij...
352153,3549.0,1578917659974,005b420fd9275f080ad79e1c4330357e,1142,41.337986,-81.402637,eyJ0aW1lc3RhbXAiOjE1Nzg5MTc2NTk5NzQsInZpbiI6Ij...
352154,3550.0,1578917668955,005b420fd9275f080ad79e1c4330357e,1142,41.338722,-81.403610,eyJ0aW1lc3RhbXAiOjE1Nzg5MTc2Njg5NTUsInZpbiI6Ij...
352155,3556.0,1578918206960,005b420fd9275f080ad79e1c4330357e,1142,41.292217,-81.409466,eyJ0aW1lc3RhbXAiOjE1Nzg5MTgyMDY5NjAsInZpbiI6Ij...
...,...,...,...,...,...,...,...
416770,3.0,1580774399978,647353ec630718a358cdfbf99a990888,1407,40.058158,-82.863375,eyJ0aW1lc3RhbXAiOjE1ODA3NzQzOTk5NzgsInZpbiI6Ij...
416771,4.0,1580774399971,75745529f421c99f0636efa2f5ba6a4e,1466,39.971760,-82.915914,eyJ0aW1lc3RhbXAiOjE1ODA3NzQzOTk5NzEsInZpbiI6Ij...
416772,5.0,1580774399978,86752012e17bbc28fb9b8f938412626a,817,39.850929,-83.081206,eyJ0aW1lc3RhbXAiOjE1ODA3NzQzOTk5NzgsInZpbiI6Ij...
416773,6.0,1580774399995,92e64fbb0355adc2c4cd663d288b7932,26,39.875714,-83.038050,eyJ0aW1lc3RhbXAiOjE1ODA3NzQzOTk5OTUsInZpbiI6Ij...


In [ ]:
"""

Converting the timestamp column in the dataframe from milliseconds into seconds.

Making a new column in the dataframe with the converted timestamp column as a datetime object.

"""

newdata['timestamp'] = newdata['timestamp']/1000
newdata['datetime'] = pd.to_datetime(newdata['timestamp'], unit = 's')

In [ ]:
## Getting first and last locations based on trip_sequence(engine cycle)
datetimeFormat = '%Y-%m-%d %H:%M:%S.%f'
timeline_df = newdata.groupby("sequence")["datetime"].agg(["first","last"])
timeline_df

## Getting the last latitude and longitude for each trip or sequence
timeline_df["latitude"] = newdata.groupby("sequence")["navigation_location_coordinate_latitude"].agg(["last"])
timeline_df["longitude"] = newdata.groupby("sequence")["navigation_location_coordinate_longitude"].agg(["last"])

x = timeline_df["last"]-timeline_df["first"]
duration = pd.to_timedelta(x.values, unit = 's').total_seconds()
timeline_df["trip_duration_s"] = duration

In [ ]:
"""
Process filtered dataframe into a dataframe grouped by vin and sequence and aggregate by first 
and last to find the start latitude and longitudes and end latitudes and longitudes.

    Returns:
    _ _ _ _ _ _ _ _ _ _ _ _ _
    Dataframe grouped by vin and sequence with four new columns for start/end latitudes and longitudes.

"""
def gen_timeline_df(df):
    timeline = df.groupby(["vin","sequence"])["timestamp"].agg(["first","last"])
    timeline["start_lat"] = df.groupby(["vin","sequence"])["navigation_location_coordinate_latitude"].agg(["first"])
    timeline["start_long"] = df.groupby(["vin","sequence"])["navigation_location_coordinate_longitude"].agg(["first"])
    timeline["latitude"] = df.groupby(["vin","sequence"])["navigation_location_coordinate_latitude"].agg(["last"])
    timeline["longitude"] = df.groupby(["vin","sequence"])["navigation_location_coordinate_longitude"].agg(["last"])
    return timeline
"""
Process filtered dataframe into a dataframe with the dwell time duration for each completed trip
(time of previous trip location with engine off to time of current trip with engine on)
    
    Returns:
    _ _ _ _ _ _ _ _ _ _ _ _ _
    Dataframe with new column for dwell time duration in seconds.

"""
def calc_dwell_duration(df):
    df['dwell time duration (seconds)'] = ''
    vins = df.index.levels[0]
    for i in vins:
        num_trips = len(df.loc[i])
        for j in np.arange(num_trips):
            if j == 0:
                ## first trip has no associated dwell time
                df.loc[i].iloc[j,6] = ''
            else:
                ## find differences in timestamps between consecutive trips (dwell time)
                if df.loc[i].iloc[j, 0] - df.loc[i].iloc[j - 1, 1] < 0:
                    df.loc[i].iloc[j,6] = ''
                else:
                    df.loc[i].iloc[j,6] = df.loc[i].iloc[j, 0] - df.loc[i].iloc[j - 1, 1]
    


In [ ]:
"""

Create a new dataframe with the original newdata dataframe using the two functions above to find 
the dwell time duration in seconds.

"""
dwell_df = gen_timeline_df(newdata)
calc_dwell_duration(dwell_df)
dwell_df.loc['123b9a7bbd7775662d3a55656dc2379b']

/shared-libs/python3.9/py/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,first,last,start_lat,start_long,latitude,longitude,dwell time duration (seconds)
sequence,,,,,,,
182,1.580768e+09,1.580770e+09,40.092665,-82.776403,40.161695,-83.146587,
1185,1.580767e+09,1.580768e+09,41.526163,-83.633261,41.570769,-83.657931,
2065,1.580774e+09,1.580774e+09,39.911455,-82.784533,39.911642,-82.810996,6190.533


In [ ]:
"""

Convert final dataframe with all necessary columns into a csv file to import into other notebooks
for model implementation.

"""
dwell_df.to_csv("final_dwell_duration.csv", index = True,  float_format='%.3f' )

,Unnamed: 0.1,timestamp,vin,sequence,navigation_location_coordinate_latitude,navigation_location_coordinate_longitude,key,datetime
0,138.0,1.578942e+09,00089af9f7d3588a888379cf0ffc53cc,733,41.492962,-81.995163,eyJ0aW1lc3RhbXAiOjE1Nzg5NDIzMzM3MTQsInZpbiI6Ij...,2020-01-13 19:05:33.713999872
1,3548.0,1.578918e+09,005b420fd9275f080ad79e1c4330357e,1142,41.335046,-81.398117,eyJ0aW1lc3RhbXAiOjE1Nzg5MTc1MzU5NjMsInZpbiI6Ij...,2020-01-13 12:12:15.963000064
2,3549.0,1.578918e+09,005b420fd9275f080ad79e1c4330357e,1142,41.337986,-81.402637,eyJ0aW1lc3RhbXAiOjE1Nzg5MTc2NTk5NzQsInZpbiI6Ij...,2020-01-13 12:14:19.973999872
3,3550.0,1.578918e+09,005b420fd9275f080ad79e1c4330357e,1142,41.338722,-81.403610,eyJ0aW1lc3RhbXAiOjE1Nzg5MTc2Njg5NTUsInZpbiI6Ij...,2020-01-13 12:14:28.954999808
4,3556.0,1.578918e+09,005b420fd9275f080ad79e1c4330357e,1142,41.292217,-81.409466,eyJ0aW1lc3RhbXAiOjE1Nzg5MTgyMDY5NjAsInZpbiI6Ij...,2020-01-13 12:23:26.960000000
...,...,...,...,...,...,...,...,...
736059,235.0,1.580774e+09,cc63c9825a40f6dc03a7ccfc6cdc7e8d,623,40.089127,-82.838978,eyJ0aW1lc3RhbXAiOjE1ODA3NzQzOTg5ODcsInZpbiI6Im...,2020-02-03 23:59:58.987000064
736060,238.0,1.580774e+09,cc63c9825a40f6dc03a7ccfc6cdc7e8d,623,40.089245,-82.838651,eyJ0aW1lc3RhbXAiOjE1ODA3NzQzOTk5OTMsInZpbiI6Im...,2020-02-03 23:59:59.992999936
736061,241.0,1.580774e+09,d8538c269b6cb7eec9a1f3b2bd8fc2d7,764,40.304051,-83.524914,eyJ0aW1lc3RhbXAiOjE1ODA3NzQzOTg4NzgsInZpbiI6Im...,2020-02-03 23:59:58.878000128
736062,242.0,1.580774e+09,d8538c269b6cb7eec9a1f3b2bd8fc2d7,764,40.304008,-83.524871,eyJ0aW1lc3RhbXAiOjE1ODA3NzQzOTk4NzksInZpbiI6Im...,2020-02-03 23:59:59.879000064


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=608d4940-9d52-4d53-8e1b-b72eb309e61f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>